### Chapter 9

+ 덕 타이핑

파이썬에서는 덕 타이핑을 이용해 사용자 정의 자료형도 내장 자료형처럼 동작하게 만들 수 있다. 단지 객체에 필요한 메서드만 구현하면, 기대한 대로 동작한다. 가령 다음과 같은 코드는 두 인수를 받아서 `+` 연산을 해서 리턴해 주는 함수를 만들어서 실험해 보는 코드이다. 실행 결과를 보면 숫자를 인수로 받았을 때도, 문자를 인수로 받았을 때도 잘 작동한다. 숫자든 문자열이든 `+` 연산이 잘 오버로딩되어 있기 때문이다.

In [1]:
def add_any(a,b):
  return a+b

n1,n2=10,20
s1,s2="김","성현"

print(add_any(n1,n2))
print(add_any(s1,s2))

30
김성현


+ 파이썬 객체의 동작 만들기

그럼 이제 클래스 객체에, 그런 메서드들을 만들어 줄 수 있는 방법들을 알아보자.

먼저 객체 표현 기법을 알아보자. 우리는 `repr`, `str` 메서드가 있다는 걸 알고 있다. `repr`은 객체를 개발자가 보고자 하는 형태로 표현한 문자열을 반환하고 `str`은 사용자가 보고자 하는 형태의 문자열을 반환한다. 만약 객체에 `__str__` 메서드가 구현되어 있지 않다면 `__repr__`이 호출된다.

또다른 방법으로는 `__bytes__`와 `__format__` 특별 메서드가 있다. `__bytes__`는 객체를 바이트 시퀀스로 표현하는 것이고 `__format__`은 특별 포맷 코드를 이용해 객체를 표현하는 문자열을 반환한다. 이러한 여러 메서드들을 살펴보기 위해 2차원 벡터를 표현하는 클래스를 만들어 보자.

In [2]:
from array import array
import math

class Vector2d:
  typecode='d' #byte 변환간에 사용하는 클래스 속성

  def __init__(self,x,y): #생성자
    self.x=float(x)
    self.y=float(y)

  def __iter__(self):
    return (i for i in (self.x,self.y))

  def __repr__(self):
    class_name=type(self).__name__
    return '{}({!r}, {!r})'.format(class_name, *self)

  def __str__(self):
    return str(tuple(self))

  def __bytes__(self):
    return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))

  def __eq__(self, other): #이는 Vector2d 객체뿐만 아니라, 동일한 숫자값을 가진 어떤 반복형 객체에 대해서든지 모두 작동한다.
    return tuple(self)==tuple(other)

  def __abs__(self):
    return math.hypot(self.x, self.y)

  def __bool__(self):
    return bool(abs(self))
  

v1=Vector2d(3,4)
print(v1)

(3.0, 4.0)


+ @classmethod 데커레이터의 사용

그런데 bytes 특별 메서드를 구현하여 객체를 이진 표현으로 나타낼 수 있게 했지만 이진 표현을 다시 객체로 만드는 메서드가 없다. 이때 array.array 가 frombytes() 라는 메서드를 가진다! 우리는 이것을 활용할 것이다. 다음과 같은 코드를 추가할 수 있다.

In [6]:
@classmethod
def frombytes(cls,octets):
  typecode=chr(octets[0])
  memv=memoryview(octets[1:]).cast(typecode)
  return cls(*memv)

Vector2d.frombytes=frombytes #Vector2d 클래스에 frombytes 메서드를 추가
byte_v1=bytes(v1)
print(byte_v1)
v2=Vector2d.frombytes(byte_v1)
print(v2)

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'
(3.0, 4.0)


위 함수는 typecode와 바이트 표현을 받은 후 그걸 이용해, 아까 인수로 받았던 클래스의 생성자를 호출해 새로운 인스턴스를 만들어 리턴한다. 기능 자체는 특별할 게 없다. 하지만 문제는 인수를 받는 방식이다. 분명 클래스의 메서드인데 self 대신 클래스 자체가 cls 매개변수로 전달되는 것이다.

이걸 위한 것이 바로 `@classmethod` 데커레이터다. classmethod 데커레이터는 객체가 아닌 클래스 자체를 첫 번째 인수로 받게 만든다. 객체가 아닌 클래스에 연산을 수행하는 메서드를 정의하는 것이다. 예시를 보자.

In [ ]:
class Test:
  num=10

  @classmethod
  def add(cls,x,y):
    return x+y

print(Test.add(10,20))

30


클래스의 인스턴스가 아니라 클래스 그 자체에 메서드를 적용한 것이다! 파이썬에서 언제나 넣어 줘야 하는 self 인수 대신 클래스 자체가 들어간다. 그럼 이런 게 왜 필요할까? 

+ @classmethod 데커레이터의 사용 예시

날짜를 저장하는 다음과 같은 간단한 클래스를 보자.

In [ ]:
class Date:
  word='date : '

  def __init__(self, date):
    self.date=self.word+date

  def now(self):
    return Date("today")

  def show(self):
    print(self.date)

a=Date("2020.08.20")
a.show()
a.now().show()


class KoreanDate(Date):
  word='날짜 : '

b=KoreanDate("2020.08.20")
b.now().show()

date : 2020.08.20
date : today
date : today


이렇게 할 경우, KoreanDate 클래스의 now함수는 KoreanDate 인스턴스 내의 word 변수를 불러오는 것이 아니라 Date 클래스에서와 똑같이 처리된다. now가 Date 객체를 만들어서 돌려주므로 당연한 일이다. 물론 KoreanDate 클래스에서 now함수를 간단하게 오버라이딩한다면 쉽게 이런 문제를 해결할 수 있다. 

하지만 만약 Date클래스를 다른 여러 클래스에 상속할 것이고 now함수를 상속한 클래스에서도 계속 사용할 거라면, 각각의 클래스에서 모두 오버라이딩을 하는 것은 별로 효율적이지 못하다.

이때 `@classmethod`를 쓸 수 있다. 그 메서드가 소속되어 있는 클래스 자체에 적용되므로 상속에 대한 걱정을 할 필요가 없다.


In [ ]:
class Date:
  word='date : '

  def __init__(self, date):
    self.date=self.word+date

  @classmethod
  def now(cls):
    return cls("today")

  def show(self):
    print(self.date)

a=Date("2020.08.20")
a.show()
a.now().show()


class KoreanDate(Date):
  word='날짜 : '

b=KoreanDate("2020.08.20")
b.now().show()

date : 2020.08.20
date : today
날짜 : today


now 함수에 `@classmethod` 데커레이터를 적용하였다. 이제 now함수는 클래스 그 자체를 인수로 받게 되므로, 상속한 클래스에서도 부모 클래스가 호출될 위험 없이 편하게 사용할 수 있다. 반면 `@staticmethod` 데커레이터 같은 경우 다른 언어의 정적 메서드와 비슷하다. 클래스에서 직접 접근하며 인스턴스별로 달라지는 것이 아니라 함께 공유한다. 모듈 대신 클래스 본체 안에 정의된 함수일 뿐인 것이다. 하지만 `@classmethod` 데커레이터가 있으므로 그다지 쓸모는 없다. 또한 클래스와 함께 작동하지 않는 함수를 정의하고 싶으면 함수를 모듈에 정의하면 될 뿐이다. 

+ 출력 포맷

이제 출력 포맷 문제에 대해서 알아본다. format() 내장 함수와 str.format() 은 실제 포맷 작업을 `__format__(format_spec)` 에 위임한다. 인수 format_spec은 포맷 명시자라고 한다. 이걸 지정하는 방법은 두 가지다. 아래 코드를 보자.

In [ ]:
brl=1/2.43
print(brl)
print(format(brl,'0.3f'))
print("{rate:0.2f}".format(rate=brl))

0.4115226337448559
0.412
0.41


format의 두 번째 인수로 주거나, str.format() 에 사용된 포맷 문자열 안에 {}로 구분한 대체 필드 내부에 있는 콜론 뒤의 문자열로 주는 방식이다. 이때 포맷 명시자에 사용된 표기법을 '포맷 명시 간이 언어' 라고 한다.

그런데 파이썬을 배운 사람이라면 {} 대체 필드 안에 포맷 명시 간이 언어를 주는 걸 한번쯤 봤을 것이다.

In [7]:
pi=3.141592
print('{:.2f}'.format(pi))

3.14


위와 같은 경우 pi변수를 소수점 둘째 자리까지 반올림해서 출력해 주는 것이다. 이런 식으로 몇몇 내장 자료형은 포맷 명시 간이 언어에 자신만의 표현 코드를 가지고 있다. int형의 경우 b, 16진수는 x, 고정 소수점의 경우 위와 같은 f등등.. 그런데 이런 식으로 내장된 자료형 외에 사용자 정의 자료형에서도 포맷 명시 간이 언어를 확장할 수 있다. 이를테면 파이썬의 내장 모듈인 datetime 모듈의 클래스들은 시, 분, 초를 출력할 수 있는 포맷을 지원한다. 이런 포맷 명시 간이 언어를 우리가 만든 벡터 클래스에도 만들어 보자. 일단 우리의 벡터 클래스에 `__format__` 메서드를 정의해 주는 것부터 시작하자.

In [8]:
def vector_format(vector, fmt_spec=''):
  components=(format(c,fmt_spec) for c in vector)
  return '({}, {})'.format(*components)

Vector2d.__format__=vector_format

v1=Vector2d(3,4)
print(format(v1, '.2f'))

(3.00, 4.00)


위와 같이 Vector2d 클래스에 format 특별 메서드를 지정해 주었다. 이때, 저런 식으로 함수 객체를 이용해서 클래스 메서드를 새로 만들어 주는 것도 가능하다. 멍키 패칭(런타임 중에 프로그램 소스를 바꾸는 것)등에 사용된다. 아무튼 이제 Vector2d 클래스도 format 함수를 사용할 수 있게 된 것이다. 그럼 Vector2d 클래스만의 포맷 명시 간이 언어를 만드는 것도 가능할까? 물론 가능하다.

그럼 벡터를 극좌표 형태로 출력해 주는 포맷 코드를 추가해 보자. 그러기 위해서는 벡터의 길이와 각도가 필요한데 크기를 생성해 주는 abs 메서드는 이미 있으므로 각을 구하는 함수만 만들어 주면 된다.

>포맷 코드를 추가할 때는 다른 자료형의 포맷 코드와 중복되지 않는 것을 사용하는 게 좋다. 정수형은 'bcdoxXn' 을, 실수형은 'eEfFgGn%'를, 문자열은 's'를 사용한다. 따라서, 우리가 극좌표 포맷 코드로 p를 선택한 건 상당히 괜찮은 선택이다. 다른 데서 중복해서 쓰이지 않기 때문이다. 물론 각 클래스에서 이 코드를 독립적으로 해석하므로 새로운 자료형에 대해 기존 포맷 코드를 재사용해도 에러는 없지만, 사용자가 혼동할 우려는 있다.

In [9]:
import math

def angle(vector):
  return math.atan2(vector.x, vector.y)

Vector2d.angle=angle

def vector_format2(vector, fmt_spec=''):
  if fmt_spec.endswith('p'): #극좌표를 출력해야 하는 경우(포맷코드 p)
    fmt_spec=fmt_spec[:-1]
    coords=(abs(vector), vector.angle())
    outer_fmt='<{}, {}>'
  else:
    coords=vector
    outer_fmt='({}, {})'

  components=(format(c,fmt_spec) for c in coords) #coord 요소들을 포맷에 맞게 출력해 준다
  return outer_fmt.format(*components)

Vector2d.__format__=vector_format2

v1=Vector2d(3,4)
print(format(v1, '.2f')) #직교좌표 출력하기
print(format(v1, '.2p')) #극좌표 출력하기

(3.00, 4.00)
<5.0, 0.64>


이런 식으로 format 특별 메서드를 만져 주면 사용자 정의 클래스에 새로운 출력 포맷 코드를 추가하는 것도 얼마든지 가능하다.

+ 해시 가능한 벡터 클래스

이제 우리가 만든 벡터 클래스를 해시 가능하게 만들어 보자. set이나 dict의 키로 사용할 수 있도록 말이다. 그러기 위해서는 Vector2d 클래스를 불변형으로 만들어 줘야 한다. 그러기 위해서 일단 x,y요소를 읽기 전용으로 만들자. `@property` 데커레이터를 이용하면 된다. 그리고 나서 해시함수를 구현하면 된다. hash 특별 메서드 문서에는 요소의 해시에 xor을 사용하는 것을 권장하므로 여기서도 그 방법을 따른다.

In [11]:
class Vector2d:
  typecode='d'

  def __init__(self,x,y):
    self.__x=float(x)
    self.__y=float(y)

  #property 데커레이터는 C#등의 언어에서의 getter와 같은 역할을 한다. @x.setter 처럼 변수명과 setter를 붙인 데커레이터는 setter의 역할을 하게 된다.
  #만약 v1.x=30 등으로 변경을 시도하면 읽기 전용 변수를 변경할 수 없기 때문에 에러가 발생한다.(can't set attribute)
  #자세한 건 19장에서 설명한다고 한다.
  @property
  def x(self):
    return self.__x

  @property
  def y(self):
    return self.__y

  def __iter__(self):
    return (i for i in (self.x, self.y))

  def __hash__(self):
    return hash(self.x)^hash(self.y)

  def __repr__(self):
    class_name=type(self).__name__
    return '{}({!r}, {!r})'.format(class_name, *self)

v1=Vector2d(3.4, 5.1)
v2=Vector2d(30, 51)
print(hash(v1))
print(hash(v2))
print(set([v1,v2]))

1152921504606845958
45
{Vector2d(30.0, 51.0), Vector2d(3.4, 5.1)}


이제 우리는 Vector2d를 이용해서 set이나 dict를 만들 수 있다.

물론 해시 가능하게 만들기 위해 굳이 프로퍼티로 getter를 만들고 객체 속성을 보호할 필요는 없다. hash와 eq 메서드를 제대로 구현하면 된다. 실험 결과 property 데커레이터 따위 없어도 해시 메서드만 있으면 잘 구현되는 게 맞다. 하지만 원칙적으로 객체의 해시값이 변하면 안 되므로 읽기 전용으로 만드는 게 좋다.

+ 파이썬의 비공개 속성과 보호된 속성

파이썬에선 private 변수를 만들 수 있는 방법이 없다. 단 서브클래스에서 속성을 실수로 변경하지 못하게 하는 메커니즘은 존재한다.

만약 `__var` 처럼 속성명에 두 개의 언더바(dunder라고도 한다)를 붙여서 정의하면 파이썬은 언더바와 클래스명을 변수명 앞에 붙여 객체 dict에 저장한다. 이를테면 class1에 `__var` 변수는 `_class1__var` 와 같은 이름으로 저장되는 것이다. 이런 파이썬 기능을 이름 장식(name mangling)이라고 한다. 이를테면 Vector2d에서 정의한 읽기 전용의 x,y변수는 `_Vector2d__x` 와 같이 저장된다.

하지만 이는 다른 이름으로 변수를 저장함으로써 안전을 제공할 뿐 실제로 변수에 접근하는 걸 원천적으로 차단하지는 못한다. 아래 코드와 같이 바뀐 이름의 변수에 직접 접근해서 변경할 경우 아무 에러도 뜨지 않고 잘 변경된다. 실수로 접근하는 건 막을 수 있지만 고의적으로 변수에 접근하는 건 막을 수 없다는 것이다.

파이썬에는 비공개 속성, 불변 속성을 정의하는 진정한 방법은 없는 것이다!

In [12]:
v1=Vector2d(3.14,2.718)
print(v1.__dict__)
v1._Vector2d__x = 30
print(v1)

{'_Vector2d__x': 3.14, '_Vector2d__y': 2.718}
Vector2d(30, 2.718)


물론 self._x 처럼 언더바 하나를 앞에 붙이면 그것을 보호된 속성으로 취급하는 것은 파이썬 프로그래머 사이에 일종의 국룰로 자리잡혀 있으므로, 언더바 두 개를 붙이는 것보다 이쪽을 더 좋아하는 사람들도 있다.

또 만약에 이름 충돌이 문제가 된다면 언더바 두 개 보다는 명시적인 변수명으로 이름을 장식하라고 말하는 사람들도 있다. 물론 본질적으로는 언더바 두 개와 같지만 다른 사람이 프로그래머의 의도를 명확히 파악할 수 있다는 것이다. 어차피 완전한 private 변수를 만들 수는 없고, 그럴 바에 프로그래머가 더 쉽게 읽을 수 있도록 하는 게 낫지 않느냐는 것이다.

+ `__slots__` 이용하기

이제 우리가 만들던 Vector2d 클래스를 더 고쳐 보자. 파이썬은 기본적으로 객체 속성을 각 객체 안의 `__dict__` 라는 딕셔너리 속성에 저장한다. 그런데 딕셔너리는 내부적으로 해시 테이블로 관리되므로, 아주 빠른 탐색 속도를 제공하는 대신 메모리 부담이 크다. 그런데 속성이 몇 개 없는 수백만 개의 객체라면, 굳이 딕셔너리를 사용할 이유는 없다. 이런 경우 `__slots__` 속성을 이용해서 메모리 사용량을 크게 줄일 수 있다. 이 속성은 파이썬 인터프리터가 객체 속성을 딕셔너리 대신 튜플에 저장하게 한다.

* 파이썬은 각 클래스에서 개별적으로 정의된 `__slots__` 속성만 고려한다. 슈퍼클래스의 slots는 서브클래스에 영향을 미치지 않는다.

다음과 같이 추가할 수 있다.

In [13]:
Vector2d.__slots__=('__x', '__y')

v1=Vector2d(1,3)
print(v1.__slots__)

('__x', '__y')


slots를 정의함으로써 '이 속성들이 이 클래스 객체가 가지는 속성' 임을 파이썬 인터프리터에 알려준다. 그럼 파이썬 인터프리터는 속성들을 각 인스턴스의 튜플 구조체에 저장한다. 이는 딕셔너리에 비해서 메모리 부담을 엄청나게 줄여준다.

단 클래스 내에 slots를 명시하는 경우 객체는 slots에 명시되지 않은 속성을 가질 수 없게 되므로 주의해야 한다. 또 slots에 `__dict__`속성을 넣게 되면 속성이 동적으로 생성될 수 있게 되므로 slots를 사용하는 의미가 없어진다.

이런 식으로 `__slots__`는 주의할 점이 많으며, 큰 테이블 형태의 데이터를 사용할 때만 적절하다. 하지만 numpy 등 그런 데이터를 처리할 때 좋은 라이브러리들이 이미 많으므로 굳이 여러 가지 주의사항을 감내해 가며 `__slot__`을 쓰는 것은 신중히 생각해야 한다.

+ 클래스 속성 오버라이드

클래스 속성을 객체 속성의 기본값으로 사용하는 것은 파이썬의 특징 중 하나다. 이를테면 우리가 만든 Vector2d 클래스에는 typecode 속성이 있다. 하지만 각각의 인스턴스에서도 클래스 속성을 변경할 수 있다. 물론 클래스 자체의 속성이 변하는 것은 아니고 그 인스턴스의 속성만 변하는 것이다.

In [ ]:
v2=Vector2d(1.1,2.2)
dumpd=bytes(v2)
print(dumpd)

print(Vector2d.typecode)
v1.typecode='f'
print(Vector2d.typecode)

dumpf=bytes(v1)
print(dumpf)

b'd\x9a\x99\x99\x99\x99\x99\xf1?\x9a\x99\x99\x99\x99\x99\x01@'
d
d
b'f\x00\x00\x80?\x00\x00@@'


위 코드를 보면 v1의 typecode가 바뀌고, typecode를 사용하는 bytes 함수의 리턴값도 변한다. 그러나 Vector2d의 typecode는 여전히 그대로다. 물론 `Vector2d.typecode='f'` 를 통해 벡터 클래스 그 자체의 typecode를 바꿀 수도 있다. 그러면 앞으로 생성하는 모든 벡터 인스턴스의 typecode가 f가 될 것이다. 하지만 클래스 속성은 공개되어 있고 모든 서브클래스가 상속하므로, 클래스 자체의 데이터 속성을 바꿀 때는 클래스를 상속하는 게 일반적이다.

In [ ]:
class ShortVector2d(Vector2d):
  typecode='f'

sv1=ShortVector2d(1.11, 3.14)
print(sv1)
print(bytes(sv1))

(1.11, 3.14)
b'f{\x14\x8e?\xc3\xf5H@'


또 클래스의 함수를 짤 때도, 만약 다른 클래스에서 상속받아서 속성을 오버라이딩해서 사용할 것이 예상된다면 클래스 속성을 사용하는 함수를 하드코딩하지 않는 것이 좋다. `type(self).__attrname__` 등으로 읽어 오는 것이 좋다. 그렇게 하면 나중에 서브클래스에서 속성을 오버라이딩하더라도, 그 속성을 사용하는 함수를 오버라이딩할 필요 없이 안전하게 사용할 수 있게 된다.